In [4]:
import pandas as pd  #  type: ignore
import numpy as np

# Cargar los datos
dataset_train = pd.read_csv(
    "../files/input/train_data.csv.zip",
    compression="zip",
)
#train_data.head()

dataset_test = pd.read_csv(
    "../files/input/test_data.csv.zip",
    compression="zip",
)
#test_data.head()

# Función para limpiar los datasets
def clean_dataset(data):
    # Renombrar la columna 'default payment next month' a 'default'
    data = data.rename(columns={"default payment next month": "default"})
    # Eliminar la columna 'ID'
    if "ID" in data.columns:
        data = data.drop(columns=["ID"])
    
    # Convertir valores no válidos a NaN
    data['EDUCATION'] = data['EDUCATION'].apply(lambda x: x if x > 0 else np.nan)
    data['MARRIAGE'] = data['MARRIAGE'].apply(lambda x: x if x > 0 else np.nan)

    # Agrupar valores mayores a 4 en la columna 'EDUCATION' como "others" (valor 4)
    data['EDUCATION'] = data['EDUCATION'].apply(lambda x: 4 if x > 4 else x)
    
    # Validación explícita: Imprimir valores únicos
    print("Valores únicos en 'EDUCATION' después de agrupar mayores a 4:")
    print(data['EDUCATION'].unique())
    
    # Eliminar registros con valores faltantes
    data = data.dropna()

    return data

# Limpiar los datasets
dataset_train = clean_dataset(dataset_train)
dataset_test = clean_dataset(dataset_test)

Valores únicos en 'EDUCATION' después de agrupar mayores a 4:
[ 3.  2.  1.  4. nan]
Valores únicos en 'EDUCATION' después de agrupar mayores a 4:
[ 2.  3.  1.  4. nan]


In [5]:
# Paso 2.
# Divida los datasets en x_train, y_train, x_test, y_test.
y_train=dataset_train.copy()
y_train=y_train.pop("default")

x_train = dataset_train.copy()
x_train.pop("default")

y_test=dataset_test.copy()
y_test=y_test.pop("default")

x_test = dataset_test.copy()
x_test.pop("default")

0       1
1       0
2       0
3       0
4       0
       ..
8995    0
8996    0
8997    0
8998    1
8999    1
Name: default, Length: 8979, dtype: int64

In [6]:
# Paso 3.
# Cree un pipeline para el modelo de clasificación. Este pipeline debe
# contener las siguientes capas:
# - Transforma las variables categoricas usando el método
#   one-hot-encoding.
# - Descompone la matriz de entrada usando PCA. El PCA usa todas las componentes.
# - Escala la matriz de entrada al intervalo [0, 1].
# - Selecciona las K columnas mas relevantes de la matrix de entrada.
# - Ajusta una red neuronal tipo MLP.

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.neural_network import MLPClassifier


#'variables_categoricas' es una lista de nombres de columnas categóricas
variables_categoricas = ["SEX", "EDUCATION", "MARRIAGE"] 
numeric_columns = [col for col in x_train.columns if col not in variables_categoricas]

# Preprocesador para las columnas categóricas
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

# Preprocesador combinado para los datos
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, variables_categoricas),                 
        ('num', StandardScaler(), numeric_columns),  
    ],
)

# Crear el pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),                         
    ('selectkbest', SelectKBest(score_func=f_classif)),
    ('pca', PCA()),                                         
    ('mlp', MLPClassifier()),
])
print("Pipeline creado con éxito.")

Pipeline creado con éxito.


In [7]:
# Paso 4.
# Optimice los hiperparametros del pipeline usando validación cruzada.
# Use 10 splits para la validación cruzada. Use la función de precision
# balanceada para medir la precisión del modelo.
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, balanced_accuracy_score

# Definir el rango de hiperparámetros para optimizar
param_grid = {
    'pca__n_components': [None],
    'selectkbest__k':[20],
    "mlp__random_state":[43],
    "mlp__hidden_layer_sizes": [(51, 31, 41)],
    'mlp__alpha': [0.26],
    "mlp__learning_rate_init": [0.001],
}



# Configurar GridSearchCV
best_model = GridSearchCV(
    estimator=pipeline,                 
    param_grid=param_grid,             
    cv=10,                              
    scoring='balanced_accuracy',                   
    n_jobs=-1,                        
    verbose=1,                                                                           
)

# Ajustar el GridSearchCV al conjunto de datos de entrenamiento
best_model.fit(x_train, y_train)

# Mostrar los mejores parámetros y el rendimiento
print("Mejores hiperparámetros encontrados:")
print(best_model.best_params_)
print(f"Mejor puntuación de validación (balanced accuracy): {best_model.best_score_:.4f}")

# Actualizar el pipeline con los mejores parámetros encontrados
best_pipeline = best_model.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits
Mejores hiperparámetros encontrados:
{'mlp__alpha': 0.26, 'mlp__hidden_layer_sizes': (51, 31, 41), 'mlp__learning_rate_init': 0.001, 'mlp__random_state': 43, 'pca__n_components': None, 'selectkbest__k': 20}
Mejor puntuación de validación (balanced accuracy): 0.6610


In [8]:
# Paso 5.
# Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz".
# Recuerde que es posible guardar el modelo comprimido usanzo la libreria gzip.

import os
import gzip
import pickle

# Guardar el objeto GridSearchCV completo
ruta_modelo = '../files/models/model.pkl.gz'
os.makedirs(os.path.dirname(ruta_modelo), exist_ok=True)
with gzip.open(ruta_modelo, 'wb') as f:
     pickle.dump(best_model, f)

In [9]:
# Paso 6.
# Calcule las metricas de precision, precision balanceada, recall,
# y f1-score para los conjuntos de entrenamiento y prueba.
# Guardelas en el archivo files/output/metrics.json. Cada fila
# del archivo es un diccionario con las metricas de un modelo.
# Este diccionario tiene un campo para indicar si es el conjunto
# de entrenamiento o prueba. Por ejemplo:
#
# {'dataset': 'train', 'precision': 0.8, 'balanced_accuracy': 0.7, 'recall': 0.9, 'f1_score': 0.85}
# {'dataset': 'test', 'precision': 0.7, 'balanced_accuracy': 0.6, 'recall': 0.8, 'f1_score': 0.75}

from sklearn.metrics import precision_score, recall_score, f1_score
import json

# Calcular las metricas
metrics = { 'type': "metrics",
    'dataset': 'train',
    'precision': precision_score(y_train, best_pipeline.predict(x_train)),
    'balanced_accuracy': balanced_accuracy_score(y_train, best_pipeline.predict(x_train)),
    'recall': recall_score(y_train, best_pipeline.predict(x_train)),
    'f1_score': f1_score(y_train, best_pipeline.predict(x_train)),
}

metrics_test = { 'type': "metrics",
    'dataset': 'test',
    'precision': precision_score(y_test, best_pipeline.predict(x_test)),
    'balanced_accuracy': balanced_accuracy_score(y_test, best_pipeline.predict(x_test)),
    'recall': recall_score(y_test, best_pipeline.predict(x_test)),
    'f1_score': f1_score(y_test, best_pipeline.predict(x_test)),}


# Guardar las métricas en formato JSON
ruta= "../files/output"
ruta_json= os.path.join(ruta, "metrics.json")

# Crear el directorio
os.makedirs(ruta, exist_ok=True)

with open(ruta_json, 'w') as f:
    json.dump(metrics, f, ensure_ascii=False) 
    f.write('\n')
    json.dump(metrics_test, f, ensure_ascii=False)  
    f.write('\n')

In [10]:

# Calcule las matrices de confusion para los conjuntos de entrenamiento y
# prueba. Guardelas en el archivo files/output/metrics.json. Cada fila
# del archivo es un diccionario con las metricas de un modelo.
# de entrenamiento o prueba. Por ejemplo:
#
# {'type': 'cm_matrix', 'dataset': 'train', 'true_0': {"predicted_0": 15562, "predicte_1": 666}, 'true_1': {"predicted_0": 3333, "predicted_1": 1444}}
# {'type': 'cm_matrix', 'dataset': 'test', 'true_0': {"predicted_0": 15562, "predicte_1": 650}, 'true_1': {"predicted_0": 2490, "predicted_1": 1420}}
#

from sklearn.metrics import confusion_matrix

# Calcular las matrices de confusión
cm_train = confusion_matrix(y_train, best_pipeline.predict(x_train))
cm_test = confusion_matrix(y_test, best_pipeline.predict(x_test))

cm_train_dict = {
    'type': 'cm_matrix',
    'dataset': 'train',
    'true_0': {'predicted_0': int(cm_train[0, 0]), 'predicted_1': int(cm_train[0, 1])},
    'true_1': {'predicted_0': int(cm_train[1, 0]), 'predicted_1': int(cm_train[1, 1])}
}

cm_test_dict = {
    'type': 'cm_matrix',
    'dataset': 'test',
    'true_0': {'predicted_0': int(cm_test[0, 0]), 'predicted_1': int(cm_test[0, 1])},
    'true_1': {'predicted_0': int(cm_test[1, 0]), 'predicted_1': int(cm_test[1, 1])}
}


#guardar en json
with open(ruta_json, 'a', encoding='utf-8') as f:
    json.dump(cm_train_dict, f, ensure_ascii=False) 
    f.write('\n')
    json.dump(cm_test_dict, f, ensure_ascii=False)  
    f.write('\n')